In [2]:
import pandas as py
from docling.document_converter import DocumentConverter

/home/ismatse/miniconda3/envs/GESTBD/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

source = "GA_61IW_615000237_1S_2024-25.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"

2025-10-25 12:41:28,031 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-25 12:41:28,048 - INFO - Going to convert document batch...
2025-10-25 12:41:28,049 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-10-25 12:41:28,056 - INFO - Loading plugin 'docling_defaults'
2025-10-25 12:41:28,057 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-25 12:41:28,063 - INFO - Loading plugin 'docling_defaults'
2025-10-25 12:41:28,065 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-25 12:41:28,076 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-10-25 12:41:28,076 - INFO - easyocr cannot be used because it is not installed.
2025-10-25 12:41:30,172 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-10-25 12:41:30,186 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-10-25 12:41:30,190 [RapidOCR] download_file.py:68: 

<!-- image -->

## PROCESO DE COORDINACIÓN DE LAS ENSEÑANZAS PR/CL/001

## ANX-PR/CL/001-01 GLYPH(cmap:df00)UÍA DE APRENDIZAJE

## ASIGLYPH(cmap:df00)NATURA

615000237 - Bases De Datos

## PLAN DE ESTUDIOS

61IW - Grado En Ingenieria Del Software

## CURSO ACADÉMICO Y SEMESTRE

2024/25 - Primer semestre

<!-- image -->

<!-- image -->

<!-- image -->

## Índice

## Guía de Aprendizaje

| 1. Datos descriptivos.........................................................................................   |
|------------------------------------------------------------------------------------------------------------------|
| 2. Profesorado................................................................................................   |
| 3. Conocimientos previos recomendados.........................................................................   |
| 4. Competencias y resultados de aprendizaje...................................................................   |
| 5. Descripción de la asi

In [4]:
# Save result.document.export_to_markdown() to a .md file
with open("output.md", "w", encoding="utf-8") as f:
    f.write(result.document.export_to_markdown())

In [6]:
import tabula

pdf_path = "output.md"

dfs = tabula.read_pdf(pdf_path, stream=True)
# read_pdf returns list of DataFrames
print(len(dfs))
dfs[0]

2025-10-25 12:52:51,575 - WARNING - 'pages' argument isn't specified.Will extract only from page 1 by default.
2025-10-25 12:52:51,701 - ERROR - Error from tabula-java:
Error: Error: Header doesn't contain versioninfo




CalledProcessError: Command '['java', '-Dfile.encoding=UTF8', '-jar', '/home/ismatse/miniconda3/envs/GESTBD/lib/python3.11/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--stream', '--guess', '--format', 'JSON', 'output.md']' returned non-zero exit status 1.

In [8]:
from pathlib import Path
import re
import pandas as pd

# Cargar el markdown completo
texto = Path("output.md").read_text(encoding="utf-8")

# Localizar la tabla de "1.1. Datos de la asignatura"
# Capturamos todo el bloque desde la línea con "| Nombre de la asignatura" hasta la línea en blanco siguiente
patron = re.compile(r"(\| Nombre de la asignatura[\s\S]+?)(?:\n\s*\n|##|\Z)")
coincidencia = patron.search(texto)

tabla_md = coincidencia.group(1) if coincidencia else None
print(tabla_md)


| Nombre de la asignatura             | 615000237 - Bases de Datos                                           |
|-------------------------------------|----------------------------------------------------------------------|
| No de créditos                      | 9 ECTS                                                               |
| Carácter                            | Obligatoria                                                          |
| Curso                               | Segundo curso                                                        |
| Semestre                            | Tercer semestre                                                      |
| Período de impartición              | Septiembre-Enero                                                     |
| Idioma de impartición               | Castellano                                                           |
| Titulación                          | 61IW - Grado en Ingenieria del Software                              |
|

In [11]:
from io import StringIO

if tabla_md:
    df = pd.read_csv(StringIO(tabla_md), sep="|", engine="python")
    # Limpieza de columnas vacías (por los bordes '|')
    df = df.dropna(axis=1, how="all")
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    # Quita posibles filas de separación con '----'
    df = df[~df.iloc[:,0].str.contains("---", na=False)].reset_index(drop=True)

    # Si la tabla está en formato clave-valor (dos columnas), convierte a dict
    if df.shape[1] == 2:
        datos = dict(zip(df.iloc[:,0], df.iloc[:,1]))
        df = pd.DataFrame([datos])

    display(df)


,No de créditos,Carácter,Curso,Semestre,Período de impartición,Idioma de impartición,Titulación,Centro responsable de la titulación,Curso académico
0,9 ECTS,Obligatoria,Segundo curso,Tercer semestre,Septiembre-Enero,Castellano,61IW - Grado en Ingenieria del Software,61 - Escuela Tecnica Superior De Ingenieria De...,2024-25


In [12]:
df.head()

,No de créditos,Carácter,Curso,Semestre,Período de impartición,Idioma de impartición,Titulación,Centro responsable de la titulación,Curso académico
0,9 ECTS,Obligatoria,Segundo curso,Tercer semestre,Septiembre-Enero,Castellano,61IW - Grado en Ingenieria del Software,61 - Escuela Tecnica Superior De Ingenieria De...,2024-25


In [13]:
from pathlib import Path
import pdfplumber
import pandas as pd
import re

def extraer_datos_asignatura_pdfplumber(pdf_path:str, titulo_regex=r"1\.1\.\s*Datos\s+de\s+la\s+asignatura"):
    """
    Busca el título en la página, recorta el área bajo el título y extrae la primera tabla.
    Devuelve un DataFrame (una fila con todas las claves/valores).
    """
    pdf_path = Path(pdf_path)
    assert pdf_path.exists(), f"No existe: {pdf_path}"

    with pdfplumber.open(pdf_path) as pdf:
        for pnum, page in enumerate(pdf.pages, start=1):
            # 1) Detectar si el título está en esta página (texto normalizado)
            text = page.extract_text() or ""
            if not re.search(titulo_regex, text, flags=re.IGNORECASE):
                continue

            # 2) Obtener posición Y del título usando palabras (para recortar por debajo)
            words = page.extract_words(use_text_flow=True, keep_blank_chars=False)
            # Normalizamos palabras
            tokens = [w["text"] for w in words]
            norm_tokens = [re.sub(r"\s+", " ", t.strip()) for t in tokens]
            joined = " ".join(norm_tokens)

            # Si falla exacto, usamos una heurística por palabras clave
            titulo_palabras = ["1.1.", "Datos", "de", "la", "asignatura"]
            idxs = []
            for i, w in enumerate(words):
                if re.sub(r"\W+", "", w["text"].lower()) == "1" or w["text"] == "1.1.":
                    # tímida búsqueda local
                    idxs.append(i)

            header_bottom = None
            # Intento 1: emparejar secuencia exacta
            lowered = [re.sub(r"\W+", "", w["text"].lower()) for w in words]
            needle = [re.sub(r"\W+", "", s.lower()) for s in titulo_palabras]
            for i in range(len(lowered) - len(needle) + 1):
                if lowered[i:i+len(needle)] == needle:
                    header_bottom = max(words[i + len(needle) - 1]["bottom"], words[i]["bottom"])
                    break

            # Intento 2: si no se encontró secuencia, recurre a buscar "1.1." y "asignatura" en la misma línea aproximada
            if header_bottom is None and words:
                line_groups = {}
                for w in words:
                    # Agrupamos por línea aproximada usando 'top' redondeado
                    key = round(w["top"] / 3)  # bucket vertical tosco
                    line_groups.setdefault(key, []).append(w)
                for line in line_groups.values():
                    texto_linea = " ".join([x["text"] for x in sorted(line, key=lambda z: z["x0"])])
                    if re.search(titulo_regex, texto_linea, flags=re.IGNORECASE):
                        header_bottom = max([x["bottom"] for x in line])
                        break

            # Si no hallamos posición, aun así prueba toda la página
            crop_top = (header_bottom + 4) if header_bottom else 0

            # 3) Recortar área por debajo del título y extraer tablas
            region = page.crop((0, crop_top, page.width, page.height))
            tablas = region.extract_tables({
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines",
                "intersection_tolerance": 5,
            })
            if not tablas:
                # fallback en modo "text" (stream)
                tablas = region.extract_tables({
                    "vertical_strategy": "text",
                    "horizontal_strategy": "text",
                })

            if not tablas:
                continue  # prueba en otra página por si el título se repite y la tabla está después

            # 4) Tomamos la primera tabla "buena" y la limpiamos
            from itertools import dropwhile
            for t in tablas:
                df = pd.DataFrame(t)
                # quitar filas separadoras '----'
                df = df[~df.iloc[:,0].fillna("").str.contains(r"^-{3,}$")]
                # si la primera fila son headers, promuévelos
                if df.shape[0] > 1 and all(isinstance(x, str) for x in df.iloc[0].tolist()):
                    # Si parece tabla clave-valor (2 columnas) no promovemos
                    if df.shape[1] > 2:
                        df.columns = df.iloc[0].fillna("").tolist()
                        df = df.drop(index=df.index[0]).reset_index(drop=True)

                # Caso típico: 2 columnas (clave | valor)
                if df.shape[1] == 2:
                    df = df.rename(columns={df.columns[0]: "clave", df.columns[1]: "valor"})
                    # Filas totalmente vacías fuera
                    df = df.dropna(how="all").applymap(lambda x: x.strip() if isinstance(x, str) else x)
                    # A DataFrame de una sola fila con todas las claves
                    kv = dict(zip(df["clave"], df["valor"]))
                    resultado = pd.DataFrame([kv])
                else:
                    # Si viniera como más columnas, te lo dejamos tal cual
                    resultado = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

                return resultado  # encontramos la tabla

    raise ValueError("No se pudo localizar la tabla de '1.1. Datos de la asignatura' en el PDF.")

# Uso:


In [14]:

df_2 = extraer_datos_asignatura_pdfplumber("GA_61IW_615000237_1S_2024-25.pdf")
df_2.head()

/tmp/ipykernel_26250/1767601265.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.dropna(how="all").applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Nombre de la asignatura,No de créditos,Carácter,Curso,Semestre,Período de impartición,Idioma de impartición,Titulación,Centro responsable de la\ntitulación,Curso académico
0,615000237 - Bases de Datos,9 ECTS,Obligatoria,Segundo curso,Tercer semestre,Septiembre-Enero,Castellano,61IW - Grado en Ingenieria del Software,61 - Escuela Tecnica Superior De Ingenieria De...,2024-25
